# Fake News Detection
### crawling data and constructing database

In [1]:
import requests
from lxml import etree
import json
import pyrebase
import csv
import pandas as pd
import numpy as np

import time
import datetime
import schedule

Outline:          
From news website homepage, crawl all news address. Next, for each news address, extract 'title', 'author', 'text' and 'url' 4 features, then send it to cloud database.



In [2]:

def getJson(siteUrl):

    # crawl json data from 'https://news.yahoo.com/'

    # the news platform homepage to crawl
    siteR = requests.get(siteUrl)
    
    # check if response succeed
    #print(siteUrl, siteR.ok) #True

    siteSel = etree.HTML(siteR.content)
    
    return siteSel

In [3]:
def getNewsList_yahoo(siteSel):

    # get the news web address list from homepage of news platform
    newsList = []

    for quote in siteSel.xpath('//div[@id="Main"]//@href'):
        
        # exclude video news
        if 'video' in quote: continue
       
        #f ix incomplete link address
        if 'https' not in quote:
            quote = 'https://news.yahoo.com/'+quote[1:]
        newsList.append(quote)

    #print('from yahoo,total',len(newsList),'urls')
    
    return newsList

In [4]:
def getNewsList_fox(siteSel):

    # get the news web address list from homepage of news platform
    newsList = []

    for quote in siteSel.xpath('//div[@class="content article-list"]//h2[@class="title"]//@href'):
       
        if 'video' in quote: continue
    
        if 'https' not in quote:
            quote = 'https://news.foxnews.com/'+quote[1:]
        newsList.append(quote)

    #print('from fox,total',len(newsList),'urls')
    
    return newsList

In [5]:
def getNewsList_huffpost(siteSel):

    # get the news web address list from homepage of news platform
    newsList = []

    for quote in siteSel.xpath('//div[@class="inner-group"]//div[@class="card__headlines"]//@href'):
    
        if 'video' in quote:continue
        
        if 'https' not in quote:
            quote = 'https://www.huffpost.com/'+quote[1:]
        newsList.append(quote)

    #print('from huffpost,total',len(newsList),'urls')
    
    return newsList


In [6]:
def getNewsList_buzzfeed(siteSel):

    # get the news web address list from homepage of news platform
    newsList = []

    for quote in siteSel.xpath('//div[@class="news-feed grid-layout-main"]//article//h2//@href'):
    
        if 'video' in quote:continue
        
        if 'https' not in quote:
            quote = 'https://www.buzzfeednews.com/'+quote[1:]
        newsList.append(quote)

    #print('from buzzfeed,total',len(newsList),'urls')
    
    return newsList



In [7]:
def getFeatures_yahoo(url):
    
    newsR = requests.get(url)
    if newsR.ok is False: return
        
    newsSel = etree.HTML(newsR.text)

    # extract features
    [title] = newsSel.xpath('//head//title/text()')
    
    author_ = newsSel.xpath('//body//article[@role="article"]//script[@type="application/ld+json"][1]/text()')
    author_ = json.loads(author_[0].strip())['author']['name']
    author = [author_]

    text=str()
    for t in newsSel.xpath('//article//div[@class="caas-body"]/p/text()'):
        t += '\n' # add linebreak if needed
        text += t

    newsFeatures = {'title':title,'author':author,'text':text,'url':url}
    
    # remove missing values 
    if '' in newsFeatures.values() or [] in newsFeatures.values():
        return None
    else:
        return newsFeatures

In [8]:

def getFeatures_fox(url):
    
    newsR = requests.get(url)
    if newsR.ok is False: return
    
    newsSel = etree.HTML(newsR.text)
    
    [title] = newsSel.xpath('//header//h1/text()')
    author = newsSel.xpath('//div[@class="author-byline"]//a/text()')
    
    text=str()
    for t in newsSel.xpath('//article//div[@class="article-body"]/p/text()'):
        t += '\n'
        text += t
    
    newsFeatures = {'title':title,'author':author,'text':text,'url':url}
    
    if '' in newsFeatures.values() or [] in newsFeatures.values():
        return None
    else:
        return newsFeatures

In [9]:
def getFeatures_huffpost(url):
    
    newsR = requests.get(url)
    if newsR.ok is False: return
        
    newsSel = etree.HTML(newsR.text)

    
    [title] = newsSel.xpath('//*[@id="entry-header"]/header//h1/text()')

    author = newsSel.xpath('//div[@class="entry__byline__author"]/a[1]/@data-vars-item-name')

    text=str()
    for t in newsSel.xpath('//*[@id="entry-body"]//div[@class="primary-cli cli cli-text "]//p/text()'):
        t += '\n' # add linebreak if needed
        text += t

    newsFeatures = {'title':title,'author':author,'text':text,'url':url}
    
    if '' in newsFeatures.values() or [] in newsFeatures.values():
        return None
    else:
        return newsFeatures

In [10]:
def getFeatures_buzzfeed(url):
    
    newsR = requests.get(url)
    if newsR.ok is False: return
        
    newsSel = etree.HTML(newsR.text)

    
    [title] = newsSel.xpath('//*[@id="buzz-content"]//article//h1/text()')

    author = newsSel.xpath('//article//span[@class="metadata-link headline-byline_bylineName__D9j7i"]/text()')

    text=str()
    for t in newsSel.xpath('//div[@class="js-subbuzz-wrapper"]//p/text()'):
        t += '\n' # add linebreak if needed
        text += t

    newsFeatures = {'title':title,'author':author,'text':text,'url':url}
    
    if '' in newsFeatures.values() or [] in newsFeatures.values():
        return None
    else:
        return newsFeatures


In [11]:
def connect2db():
    # connect to cloud dastabase
    # initialize cloud database 'fakenews'

    config = {"apiKey": "apiKey",
              "authDomain": "projectId.firebaseapp.com",
              "databaseURL": "https://fakenews-53e87-default-rtdb.firebaseio.com/",
              "storageBucket": "projectId.appspot.com",
              "serviceAccount": "fakenews-53e87-firebase-adminsdk-3wmt8-e668bd4ef8.json"
              }
    firebase = pyrebase.initialize_app(config)
    db = firebase.database()
    return db


In [12]:
def openRecords():
    # create records of id-url pairs of news

    f = open('newsURLs.csv','a+',encoding='utf-8')
    writer = csv.writer(f)
    newsURLs = pd.read_csv('newsURLs.csv')
    print('number of news now:',len(newsURLs))

    return newsURLs

In [13]:
def main():
    
    
    print('----------------')
    print(datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S'))
    
    # connect to cloud dastabase and initialize cloud database
    db = connect2db()
    # open newsURLs.csv for recording
    newsURLs = openRecords()
    
    # news websites we focus on
    sites = ['https://www.yahoo.com/',
             'https://www.foxnews.com/',
             'https://www.huffpost.com/',
             'https://www.buzzfeednews.com/']

    for siteUrl in sites:

        try:

            siteSel = getJson(siteUrl)
            # get news list
            if siteUrl == 'https://www.yahoo.com/':
                newsList = getNewsList_yahoo(siteSel)
            elif siteUrl == 'https://www.foxnews.com/':
                newsList = getNewsList_fox(siteSel)
            elif siteUrl == 'https://www.huffpost.com/':
                newsList = getNewsList_huffpost(siteSel)
            elif siteUrl == 'https://www.buzzfeednews.com/':
                newsList = getNewsList_buzzfeed(siteSel)
        except Exception as e:
            print("error:", e)
            continue

        # put features of each news to cloud database
        for url in newsList:

            try:
                # delete dubplicates from newsList and append it to newsURLs.csv
                f = open('newsURLs.csv', 'a+', encoding='utf-8')
                writer = csv.writer(f)

                # remove url if duplicate with urls before
                newsURLs = pd.read_csv('newsURLs.csv')
                # print('now the records have total',len(newsURLs))

                if url in newsURLs['url'].values:
                    newsList.remove(url)
                    continue

                # get features of each news
                # newsFeatures = {'title':title,'author':author,'text':text}
                if siteUrl == 'https://news.yahoo.com/':
                    newsFeatures = getFeatures_yahoo(url)
                elif siteUrl == 'https://www.foxnews.com/':
                    newsFeatures = getFeatures_fox(url)
                elif siteUrl == 'https://www.huffpost.com/':
                    newsFeatures = getFeatures_huffpost(url)
                elif siteUrl == 'https://www.buzzfeednews.com/':
                    newsFeatures = getFeatures_buzzfeed(url)


                # save data to cloud database when newsFeatures is non-empty
                if newsFeatures is not None:
                    # write distinct id in newsURLs.csv for records
                    i = len(newsURLs)
                    writer.writerow([i, url])
                    f.close

                    db.child("news").child(i).set(newsFeatures)

            except Exception as e:
                print("error:", e)
                continue

    return



In [ ]:
# execute programe every 30 minutes, to enable data flow

main()

schedule.every(30).minutes.do(main)
while True:
    schedule.run_pending()
    time.sleep(600)

----------------
2022/04/25 22:41:21
number of news now: 337
error: not enough values to unpack (expected 1, got 0)
